In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import transformers
from transformers import LayoutLMForTokenClassification,\
    LayoutLMTokenizer, AdamW, LayoutLMv2Processor, LayoutLMv2ForTokenClassification
# from tensordict import TensorDict
from datasets import load_dataset
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import ImageDraw, Image
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import logging
# from torchvision.transforms import PILToTensor

import os 
if 'notebooks' in os.getcwd():
    os.chdir("..")

/home/pedro/Desktop/Polytechnique/few-shot-learning-transformers/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
logging.basicConfig(filename='logs/train.log', encoding='utf-8', level=logging.DEBUG)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cpu')

In [6]:
dataset = load_dataset("katanaml/cord", )

# dataset = load_dataset("darentang/sroie")

Repo card metadata block was not found. Setting CardData to empty.


In [7]:
dataset['train'][0]['ner_tags']

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 5,
 1,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 5,
 1,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 5,
 1,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 1,
 3,
 3,
 5,
 1,
 3,
 3,
 3,
 5,
 13,
 13,
 12,
 12,
 14,
 14,
 11,
 11,
 22,
 22,
 22]

## Creating PyTorch Datasets, DataLoader

In [8]:
processor =LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")


In [9]:
class SROIE(Dataset):
    def __init__(self, 
                 data,
                 encode : bool = False) -> None:
        super().__init__()

        if encode:
            self.X = []
            for example in tqdm(data):
                X= SROIE.encode(example)
                self.X.append(X)

        else:
            self.X = data

    @staticmethod
    def encode(
        example, 
        max_seq_length = 512, 
        pad_token_box = [0,0,0,0]
    ):
        words = example['words']
        boxes = example['bboxes']
        image = Image.open(example['image_path'])
        word_labels = example['ner_tags']

        encoded_inputs = processor(
            image, 
            words, 
            boxes=boxes, 
            word_labels=word_labels, 
            padding="max_length", 
            truncation=True, 
            return_tensors="pt"
        )

        # token_boxes = []
        # for word, box in zip(words, bboxes):
        #     word_tokens = tokenizer.tokenize(word)
        #     token_boxes.extend([box] * len(word_tokens))

        
        # special_tokens_count = 2 
        # if len(token_boxes) > max_seq_length - special_tokens_count:
        #     token_boxes = token_boxes[: (max_seq_length - special_tokens_count)]

        # # add bounding boxes of cls + sep tokens
        # token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]

        # encoding = tokenizer(' '.join(words), padding='max_length', truncation=True)
        # # Padding of token_boxes up the bounding boxes to the sequence length.
        # input_ids = torch.tensor(tokenizer(' '.join(words), truncation=True)["input_ids"])
        # padding_length = max_seq_length - len(input_ids)
        # token_boxes += [pad_token_box] * padding_length

        # encoding['labels'] = torch.tensor(example['ner_tags'])
        # encoding['bbox'] = torch.tensor(token_boxes)
        # # encoding['input_ids'] = torch.tensor(encoding['input_ids'])
        # encoding['token_type_ids'] = torch.tensor(encoding['token_type_ids'])
        # encoding['attention_mask'] = torch.tensor(encoding['attention_mask'])
        
        return encoded_inputs
    
    def __getitem__(self, index: int):
        return self.X[index]

    def __len__(self):
        return len(self.X)

In [10]:
data = []
for example in tqdm(dataset['train']):
    words = example['words']
    boxes = example['bboxes']
    image = Image.open(example['image_path']).convert("RGB")
    word_labels = example['ner_tags']

    try:
        encoded_inputs = processor(
            image, 
            words, 
            boxes=boxes, 
            word_labels=word_labels, 
            padding="max_length", 
            truncation=True, 
            return_tensors="pt"
        )

        data.append(encoded_inputs)
    except:
        pass

    # assert encoded_inputs.input_ids.shape == torch.Size([512])
    # assert encoded_inputs.attention_mask.shape == torch.Size([512])
    # assert encoded_inputs.token_type_ids.shape == torch.Size([512])
    # assert encoded_inputs.bbox.shape == torch.Size([512, 4])
    # assert encoded_inputs.image.shape == torch.Size([3, 224, 224])
    # assert encoded_inputs.labels.shape == torch.Size([512]) 

  0%|          | 0/800 [00:00<?, ?it/s]

100%|██████████| 800/800 [00:59<00:00, 13.40it/s]


In [11]:
encoded_inputs.input_ids.shape

torch.Size([1, 512])

In [12]:
df = SROIE(data, encode=False)

In [13]:
dataloader = DataLoader(
    df,
    shuffle=True,
    batch_size= 2
)

In [14]:
example = dataset['train'][0]

In [15]:
unique_rows = []
for row in dataset['train']['ner_tags']:
    unique_rows.append(np.unique(row))

In [16]:
n_labels = np.unique(np.concatenate(unique_rows)).shape[0]

In [17]:
n_labels

23

## Importing model

In [18]:

model = LayoutLMv2ForTokenClassification.from_pretrained(
    'microsoft/layoutlmv2-base-uncased',
    num_labels=n_labels
)
model.to(device)

Some weights of LayoutLMv2ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv2-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LayoutLMv2ForTokenClassification(
  (layoutlmv2): LayoutLMv2Model(
    (embeddings): LayoutLMv2Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (visual): LayoutLMv2VisualBackbone(
      (backbone): FPN(
        (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

### Number of treinable / non-treinable parameters

In [54]:
print(f'''
    Model Info
    -----------------
    
    Treinable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}
    Non Treinable params: {sum(p.numel() for p in model.parameters() if not p.requires_grad)}

''')


    Model Info
    -----------------
    
    Treinable params: 199673750
    Non Treinable params: 425152




In [19]:
optimizer = AdamW(model.parameters(), lr=5e-5)

global_step = 0
num_train_epochs = 4
t_total = len(dataloader) * num_train_epochs # total number of training steps 

#put the model in training mode
model.train()
for epoch in tqdm(range(num_train_epochs)):
  logging.info(f"Epoch: {epoch}")
  running_loss = 0.0
  correct = 0
  for X in tqdm(dataloader):
      input_ids = X["input_ids"]\
         .to(device)\
         .squeeze()
      
      bbox = X["bbox"]\
         .to(device)\
         .squeeze()

      attention_mask = X["attention_mask"]\
         .to(device)\
         .squeeze()
      token_type_ids = X["token_type_ids"]\
         .to(device)\
         .squeeze()
      
      labels = X["labels"].to(device)
      image = X["image"]\
         .to(device)\
         .squeeze()

      # forward pass
      outputs = model(
        input_ids=input_ids, 
        bbox=bbox, 
        attention_mask=attention_mask, 
        token_type_ids=token_type_ids,
        image= image,
        labels=labels
      )
      loss = outputs.loss

      running_loss += loss.item()
      predictions = outputs.logits.argmax(-1)
      correct += (predictions == labels).float().sum()

      # backward pass to get the gradients 
      loss.backward()

      # update
      optimizer.step()
      optimizer.zero_grad()
      global_step += 1
  
  logging.info("Loss:", running_loss / X["input_ids"].shape[0])
  accuracy = 100 * correct / len(data)
  logging.info("Training accuracy:", accuracy.item())

/home/pedro/Desktop/Polytechnique/few-shot-learning-transformers/.conda/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [03:46<?, ?it/s]


KeyboardInterrupt: 

In [107]:
outputs = model(
        input_ids=input_ids.squeeze(), 
        bbox=bbox, 
        attention_mask=attention_mask.squeeze(), 
        token_type_ids=token_type_ids.squeeze(),
        image= image,
        labels=labels
      )

In [77]:
input_ids.shape

torch.Size([2, 1, 512])

In [78]:
bbox.shape

torch.Size([2, 512, 4])

In [79]:
attention_mask.shape

torch.Size([2, 1, 512])

In [80]:
token_type_ids.shape

torch.Size([2, 1, 512])

In [81]:

image.shape

torch.Size([2, 3, 224, 224])

In [86]:
torch.unique(labels)


tensor([-100,    1,    3,    5,    9,   10,   13,   14,   15,   16,   22])